# Golf Gamer

## Final Project - Alan Ranciato

![Image of Augusta National](https://golf.com/wp-content/uploads/2020/04/augusta-national-12-996x560.jpg)

## Project Description
This project is a golf contest wagering contract to be used for small to large groups.

The contract tracks multiple players, their number of skins and CTPs (closest to the pin) won as well as their total scores for multiple rounds of golf.  After requiring a buyin from each player, the contract calculates the pro-rated payout per contest for each player, calculates a commission (to pay for both gas and administration), and provides payouts to the players and the owner of the contract.

Along the way, the contract allows for changing the percentage of payout for each game, the number of score attestations needed to finalize the game, and the ability to see scores and contest standings along the way.  

At the end, when the withdraw() function is called, final payouts are calculated (total won - commission), players are all paid, the contract remaining balance is paid to the owner of the contract and events are emited to the blockchain.

The steps to run and validate the contract are illustrated below


###  Copy of the Final Project Proposal 

1) Type of app

This will be a DApp.  I plan to implement with a javascript front end, but due to limited experience with it, may stick to Jupyter (will definitely do a notebook to test and submit along side.)

2) Main Idea

DApp for managing a golf tournament and prizes. Will calculate different games (skins, closest to the pin, etc) and payouts in coin (GOLF coin is already in existance.  Might need to come up with something else via ICO, or accept multiple forms of payment / coins)

3) Potential Market

Golfers and gamers.  The intent is to use for small tournaments and golf trips between friends, but could see it as being a service offered and take a "house percentage".

4) Logistics

Contract written in Solidity.  Jupyter Notebook for testing and Demo.  Potentially - web front-end interface.  Will likely demo with jupyter via Youtube so I can show the outcomes of the data as well.  Testing via Remix. 

5) Last Mile Problem

Biggest issue is the oracle problem. The data entry will likely always be manual as it's golf and not a sanctioned professional event. In order to solve - will build in capability to specify the number of votes needed for consensus.

6) Scaling issues

I don't know that scaling would be too big of an issue.  Obviously managing the front-end capacity would be a thing.  Nothing should be too time sensitive.  I think the biggest issue could be a rise in cost of gas for the blockchain and pricing out usage.

7) Fraud and Malicious behavior

Inputting incorrect scores could be an issue - hence the reason for consensus and the ability to update.  Also, as golfers are not traditionally the most technical of people, I can see kay management being an issue.

8) Compare without blockchain

The solution could definitley be done without blockchain as a centralized service.  I think what blockchain provides is the ability to manage contracts and tournaments yourself vs relying on a service.  Additionally, the gambling aspect is now just a friendly game between friends vs. a centralized service that might be regulated.  





### Basic housekeeping and configuration before the good stuff starts


#### Setup the address of the contract in Ganache.  This is the one part of this notebook that needs to be configured

The contract has been deployed to Ganache via Truffle migrate.  The constructor of the contract has the following inputs (and values being used for this deployment test):

* Players (addresses) : players 1-4 from the test accounts provided in Ganache 
* Entry fee (in ETH) : 20 - conversion to wei is done within the contract
* Attestors: 1 - how many attestors are needed to validate results - the Last Mile problem
* Commission Percent: 3 - this is the administration fee for running the contract and gets paid to the owner after all players are paid


In [ ]:
#Setting the contract Address for Ganache
GOLFGAMER_CONTRACT_ADDR = "0xc808Ff8bD6548E877a4104E94B40F82Ef6D7AE6b"

#### Setup solc / ganache

In [ ]:
from solcx import install_solc, set_solc_version, compile_files
from web3 import Web3
from decimal import *

In [ ]:
install_solc('v0.7.6')
set_solc_version('v0.7.6')

In [ ]:
GANACHE="http://127.0.0.1:7545"

In [ ]:
# web3.py instance
w3 = Web3(Web3.HTTPProvider(GANACHE))
w3.eth.accounts


In [ ]:
main_account = w3.eth.accounts[0]
main_account

In [ ]:
# set default account on web3 object
w3.eth.defaultAccount = main_account
w3.eth.defaultAccount

In [ ]:
# construct path to compiled contract file
def get_compiled_contract_path(contract_name):
    return "./build/contracts/"+contract_name+".json"

In [ ]:
CONTRACT_NAME = "GolfGamer"

In [ ]:
import json

golfgamer_compiled_path = get_compiled_contract_path(CONTRACT_NAME)
golfgamer_compiled = json.load(open(golfgamer_compiled_path))
golfgamer_abi = golfgamer_compiled['abi']
golfgamer_bytecode = golfgamer_compiled['bytecode']
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)

We'll make sure we can see all of the public functions from our contract

In [ ]:
golfgamer_contract.all_functions()

#### Build out Utility functions to make life easier for testing

In [ ]:
# execute a call, which does not execute a transaction (i.e. no write => no gas)
def exec_call(w3, contract_inst, function_name, *f_args):
    func_inst = contract_inst.get_function_by_name(function_name)

    return_value = func_inst(*f_args).call()
    return return_value

In [ ]:
# execute a transaction (i.e. a write), and return the transaction receipt (costs gas paid in Ether)
def exec_transact_receipt(w3, contract_inst, function_name, *f_args):
    func_inst = contract_inst.get_function_by_name(function_name)
    
    # get the return value, without executing transaction
    return_value = exec_call(w3, contract_inst, function_name, *f_args)
    
    # execute the transaction
    tx_hash = func_inst(*f_args).transact()
    # receipt does not contain values returned by function
    tx_receipt = w3.eth.waitForTransactionReceipt(tx_hash)
    
    return return_value, tx_receipt

In [ ]:
# execute transaction, but ignore the receipt
def exec_transact(w3, contract_inst, function_name, *f_args):
    rv, _ = exec_transact_receipt(w3, contract_inst, function_name, *f_args)
    return rv

### Testing the Golf Gamer contract

#### Let's start by finding out some information about the players and payouts

We'll create a similar status enum for easier mapping in the notebook - This enum maps to the statuses of the contract.

In [ ]:
import enum
class Statuses(enum.Enum):
   OPEN = 0
   PENDINGATTEST = 1
   CLOSED = 2   
   CLOSEDPAID = 3

Here we'll pull the players from the contract construction and set variables for easier access throughout

In [ ]:
players = exec_call(w3, golfgamer_contract, 'getPlayers')
player1 = players[0]
player2 = players[1]
player3 = players[2]
player4 = players[3]
players

In [ ]:
player2

The initial status of the contract should be OPEN.  Let's verify that here

In [ ]:
status = Statuses(exec_call(w3, golfgamer_contract, 'getStatus')).name
assert status == 'OPEN'
status

Here we'll look to see what the payout values are currently based on how many skins / ctps have been won.  Since no scores have been entered yet, we're expecting 0 amounts

In [ ]:
w3.fromWei(exec_call(w3, golfgamer_contract, 'getSkinsValue'),'ether')

In [ ]:
w3.fromWei(exec_call(w3, golfgamer_contract, 'getCtpValue'), 'ether')

The total pot should consist of the number of players (4) * the entry fee (20)

In [ ]:
totalPot = w3.fromWei(exec_call(w3, golfgamer_contract, 'getTotalPot'), 'ether')
assert totalPot == 80
totalPot

The getPayouts function returns us the payouts based on the percentage of each game * the total pot.  Note that this does not calculate in the commission percentage. That is only calculated at withdrawl.  Note that we are /100 on the payouts due to the decimal conversion for %s.

The default payouts at contract creation are set at 33/33/34%.  These can be changed as show in the next section.

In [ ]:
payouts = exec_call(w3, golfgamer_contract, 'getPayouts')
print ("Skins: ", w3.fromWei(payouts[0],'ether')/100 ,  ' | ' , "CTP: ", w3.fromWei(payouts[1],'ether')/100 , ' | ' , "Overall Winner: ", w3.fromWei(payouts[2],'ether')/100)

#### Now, we're going to edit the payout percentages and check the payouts again.  They should have changed from the default 33%, 33%, 34% values to the values below

Let's be sure that we validate that the percentages add up to 100

In [ ]:
try:
    exec_transact(w3, golfgamer_contract, 'setPayouts', 20, 23, 40) 
except Exception as e:
    print ("Failed as designed: \n", e)
    

Now we'll set the payouts a little differently, but in an amount that adds to 100.

In [ ]:
exec_transact(w3, golfgamer_contract, 'setPayouts', 20, 23, 57) 

In [ ]:
payouts = exec_call(w3, golfgamer_contract, 'getPayouts')
print ("Skins: ", w3.fromWei(payouts[0],'ether')/100 ,  ' | ' , "CTP: ", w3.fromWei(payouts[1],'ether')/100 , ' | ' , "Overall Winner: ", w3.fromWei(payouts[2],'ether')/100)

#### Now, let's setup some scores and enter them into the contract

In [ ]:
exec_transact(w3, golfgamer_contract, 'postScore', player1, 2, 1, 87) 
exec_transact(w3, golfgamer_contract, 'postScore', player1, 1, 0, 82) 
exec_transact(w3, golfgamer_contract, 'postScore', player2, 2, 1, 91) 
exec_transact(w3, golfgamer_contract, 'postScore', player2, 3, 2, 97) 
exec_transact(w3, golfgamer_contract, 'postScore', player3, 4, 0, 86) 
exec_transact(w3, golfgamer_contract, 'postScore', player3, 2, 0, 102) 
exec_transact(w3, golfgamer_contract, 'postScore', player4, 1, 0, 89) 
exec_transact(w3, golfgamer_contract, 'postScore', player4, 5, 1, 78) 


Now that we have scores posted, let's check the payouts

In [ ]:
payouts = exec_call(w3, golfgamer_contract, 'getPayouts')
overallWinner = w3.fromWei(payouts[2],'ether')/100
print ("Skins: ", w3.fromWei(payouts[0],'ether')/100 ,  ' | ' , "CTP: ", w3.fromWei(payouts[1],'ether')/100 , ' | ' , "Overall Winner: ", w3.fromWei(payouts[2],'ether')/100)

Skins Payout based on the number won - with the current scores posted, there are 20 skins at 16 eth == .8 per skin won.

In [ ]:
skinValue = w3.fromWei(exec_call(w3, golfgamer_contract, 'getSkinsValue'),'ether')/100
skinValue

CTP Payout based on number won

In [ ]:
ctpValue = w3.fromWei(exec_call(w3, golfgamer_contract, 'getCtpValue'), 'ether')/100
ctpValue

And now let's check the player scores and their winnings

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player1)
print ("Player 1: ", player1," Winnings: ", w3.fromWei(score[4],'ether'), " OverallScore: ", score[2])

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player2)
print ("Player 2: ", player2, "Winnings: ", w3.fromWei(score[4],'ether'), " OverallScore: ", score[2])

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player3)
print ("Player 3: ", player3,"Winnings: ", w3.fromWei(score[4],'ether'), " OverallScore: ", score[2])

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player4)
print ("Player 4: ", player4, "Winnings: ", w3.fromWei(score[4],'ether'), " OverallScore: ", score[2])

#### Now that the scores are all posted, we'll close the game and pay our players

Close the game so no more scores can be posted

In [ ]:
status = exec_transact(w3, golfgamer_contract, 'closeGame')

Let's make sure we can't add anymore scores now that the game is closed

In [ ]:
try:
    exec_transact(w3, golfgamer_contract, 'postScore', player4, 4, 1, 78) 
except Exception as e:
    print("This should have failed as we're not allowing any additional score entry when the status is not OPEN \n", e)

Since we constructed the contract to require at least one attestation, the status should be pending attest vs. closed

In [ ]:
assert Statuses(status).name == 'PENDINGATTEST'
Statuses(status).name

Since we need to attest the scores prior to paying out and closing the game, we'll call the attest function which will add to the attestors and update the status of the game to Closed

In [ ]:
remainingAttestations = exec_transact(w3, golfgamer_contract, 'attestGame')

In [ ]:
assert  remainingAttestations == 0
remainingAttestations

Let's see who the attestors are

In [ ]:
exec_call(w3, golfgamer_contract, 'getAttestors')

Now that we've attested to the scores, the status of the game should be CLOSED

In [ ]:
status = Statuses(exec_call(w3, golfgamer_contract, 'getStatus')).name
assert status == 'CLOSED'
status

#### Now that we've attested, we'll try to payout the players

We'll test the withdrawl, which should fail since we haven't made any deposits just yet

In [ ]:
try:
    exec_transact(w3, golfgamer_contract, 'withdraw')
except Exception as e:
    print ("Failed as designed: \n", e)

We'll create some transaction details to pass into the deposits.  This is the from address of each player as well as their entry fees.

In [ ]:
transaction_dict1 = {
     'from': player1,
    'value': w3.toWei(20, 'ether') # send this much ether as part of transaction
}
transaction_dict2 = {
     'from': player2,
    'value': w3.toWei(20, 'ether') # send this much ether as part of transaction
}
transaction_dict3 = {
     'from': player3,
    'value': w3.toWei(20, 'ether') # send this much ether as part of transaction
}
transaction_dict4 = {
     'from': player4,
    'value': w3.toWei(20, 'ether') # send this much ether as part of transaction
}

Now we'll make the deposits to fund the contract.

Check the balance -  should start at 0

In [ ]:
balance = exec_call(w3, golfgamer_contract, 'getContractBalance')
assert balance == 0
balance

We need to switch accounts to send some deposits to the contract now

In [ ]:
w3.default_account = w3.eth.accounts[1]
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)
golfgamer_contract.functions.deposit().transact( transaction_dict1 )


In [ ]:
w3.default_account = w3.eth.accounts[2]
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)
golfgamer_contract.functions.deposit().transact( transaction_dict2 )


In [ ]:
w3.default_account = w3.eth.accounts[3]
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)
golfgamer_contract.functions.deposit().transact( transaction_dict3 )


In [ ]:
w3.default_account = w3.eth.accounts[4]
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)
golfgamer_contract.functions.deposit().transact( transaction_dict4 )


And now our balance should be 80

In [ ]:
balance = exec_call(w3, golfgamer_contract, 'getContractBalance')
assert balance == w3.toWei(80,'ether')
w3.fromWei(balance, 'ether')

#### Now that we've deposited the entry fees and closed the game, it's time to payout the players (and the contract owner)

In [ ]:
w3.default_account = w3.eth.accounts[0]
golfgamer_contract= w3.eth.contract(address=GOLFGAMER_CONTRACT_ADDR, bytecode=golfgamer_bytecode, abi=golfgamer_abi)

try:
    exec_transact(w3, golfgamer_contract, 'withdraw')
except Exception as e:
    print ("Error with payouts: \n", e)


Finally, let's check the status - which should now be CLOSEDPAID

In [ ]:
status = Statuses(exec_call(w3, golfgamer_contract, 'getStatus')).name
assert status == 'CLOSEDPAID'
status

And Finally, our contract balance should be back to Zero

In [ ]:
balance = exec_call(w3, golfgamer_contract, 'getContractBalance')
assert balance == w3.toWei(0,'ether')
w3.fromWei(balance, 'ether')


### Final wrap up and validation of the balances

Lastly, let's check the balances on the accounts we used.  Account 0 was the contract owner and should have received 3% of the contract value.  The other players should have received payouts based upon their winnings.


In [ ]:
contractOwner = w3.fromWei(w3.eth.getBalance(w3.eth.accounts[0]), 'ether')
contractOwner

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player1)
winnings = (w3.fromWei(score[4],'ether')) * Decimal('.97')
print ("Player 1: ", player1," Winnings: ", winnings, "Balance: ", w3.fromWei(w3.eth.getBalance(player1), 'ether'))

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player2)
winnings = (w3.fromWei(score[4],'ether')) * Decimal('.97')
print ("Player 2: ", player2," Winnings: ", winnings, "Balance: ", w3.fromWei(w3.eth.getBalance(player2), 'ether'))

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player3)
winnings = (w3.fromWei(score[4],'ether')) * Decimal('.97')

print ("Player 3: ", player3," Winnings: ", winnings, "Balance: ", w3.fromWei(w3.eth.getBalance(player3), 'ether'))

Note that Player 4 also has the low total which is an additional 45.6 eth in addition to their 8.48 winning below.  The balance should reflect that

In [ ]:
score = exec_call(w3, golfgamer_contract, 'getScore', player4)
winnings = (w3.fromWei(score[4],'ether') + overallWinner) * Decimal('.97')
print ("Player 4: ", player4," Winnings: ", winnings, "Balance: ", w3.fromWei(w3.eth.getBalance(player4), 'ether'))

# And that's it!  Happy Golfing!  Fairways and Greens!

![Closing Course Image](https://images.ctfassets.net/56u5qdsjym8c/3b96eGN9KodYhSYaBsYpI/261024d195ef6803bca98d4e10fd2793/Blue-Doral-Monster-Hero.jpg?fl=progressive&q=80)


